In [67]:
import os
import face_recognition
from PIL import Image
from tqdm import tqdm
import math

In [3]:
base_dirs = [f"organized_data/{label}" for label in ["spoof","live"]]
print(base_dirs)

['organized_data/spoof', 'organized_data/live']


In [30]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [41]:
base_dirs = [f"organized_data/{label}" for label in ["spoof","live"]]

crop_dir = 'cropped_data'
spoof_cropped_dir = os.path.join(crop_dir, 'spoof')
live_cropped_dir = os.path.join(crop_dir, 'live')
target_dirs = {
    "organized_data/spoof":spoof_cropped_dir,
    "organized_data/live":live_cropped_dir,
}

ensure_dir(spoof_cropped_dir)
ensure_dir(live_cropped_dir)

In [33]:
#show print_count number of example of cropping the face out
print_count = 10
for base_dir in base_dirs:
    for root, dirs, files in os.walk(base_dir, topdown=True):
        for file in files:
            if print_count>0:
                print_count-=1
                path = os.path.join(root, file)
                image = face_recognition.load_image_file(path)
                face_locations = face_recognition.face_locations(image)
                
                if face_locations:
                    top, right, bottom, left = face_locations[0]
                    print(top,right,bottom,left)
                    face_image = image[top:bottom, left:right]
                    pil_image = Image.fromarray(face_image)
                    pil_image.show()

142 349 409 82
142 319 409 52
110 339 239 210
201 386 387 201
284 304 469 118
142 409 409 141
170 455 491 134


In [38]:
#find the min dimensions out of all pictures
min_dim1 = -1
min_dim2 = -1
for base_dir in base_dirs:
    for root, dirs, files in os.walk(base_dir, topdown=True):
        for file in tqdm(files):
            path = os.path.join(root, file)
            image = face_recognition.load_image_file(path)
            dim1 = image.shape[0]
            dim2 = image.shape[1]
            
            if min_dim1<0 or dim1<min_dim1:
                min_dim1 = dim1
            if min_dim2<0 or dim2<min_dim2:
                min_dim2 = dim2
print(min_dim1)
print(min_dim2)

100%|████████████████████████████████████| 19923/19923 [03:06<00:00, 106.94it/s]

85
85


In [59]:
def crop_image(min_dim1,min_dim2,image):
    #print(image.shape)
    dim1,dim2 = image.size
    offset1 = (dim1-min_dim1)//2
    offset2 = (dim2-min_dim2)//2
    cropped_image = image.crop((offset1,offset2,offset1+min_dim1,offset2+min_dim2))
    #cropped_image = image[offset1:offset1+min_dim,offset2:offset2+min_dim]
    return cropped_image

In [81]:
def crop2_image(min_dim1,min_dim2,image):
    dim1,dim2 = image.size
    ratio = max(min_dim1/dim1,min_dim2/dim2)
    new_dims = (math.ceil(dim1*ratio),math.ceil(dim2*ratio))
    image.thumbnail(new_dims)
    cropped_image = image.crop((0,0,min_dim1,min_dim2))
    
    return cropped_image

In [82]:
def crop_files(source_dir, target_dir):
    for root, dirs, files in os.walk(source_dir):
        for file in tqdm(files):
            path = os.path.join(root, file)
            target_path = os.path.join(target_dir, file)
            if not os.path.exists(target_path):
                image = Image.open(path)
                crop2_image(min_dim1,min_dim1,image)
                image.save(target_path)

In [83]:
min_dim = min(min_dim1,min_dim2)
for base_dir in target_dirs:
    target_dir = target_dirs[base_dir]
    if base_dir=="organized_data/live":
        crop_files(base_dir,target_dir)

100%|█████████████████████████████████████| 19923/19923 [03:38<00:00, 90.98it/s]


In [84]:
min_dim = min(min_dim1,min_dim2)
for base_dir in target_dirs:
    target_dir = target_dirs[base_dir]
    if base_dir=="organized_data/spoof":
        crop_files(base_dir,target_dir)

100%|█████████████████████████████████████| 47247/47247 [10:25<00:00, 75.57it/s]
